## Partie 1: algorithme naïf

In [1]:
def subset_sum(S, t):
    '''
    Algorithme naïf pour calculer la solution d'un sac à dos.
    '''
    # On utilise la bijections entre l'intervale [0, 2^#S[ et les sous-ensembles de S
    for s in range(2**len(S)):
        subset = [x for i, x in enumerate(S) if s & 2**i]
        if t == sum(subset):
            return True, subset
    return False

In [2]:
S = [11, 23, 3, 27, 16, 6, 5, 3]
subset_sum(S, 22)

(True, [16, 6])

In [3]:
subset_sum(S, 10)

False

In [4]:
subset_sum(S, 56)

(True, [23, 27, 6])

## Partie 2: algorithme récursif

In [5]:
def partial_subset_sum(S, i, t):
    '''
    Solution d'un subset-sum tronqué au i-ème élément
    '''
    # Si on considère la sous-liste vide, on ne peut obtenir que 0
    if i < 0:
        return 0
    # La meilleure solution ne contenant pas S[i]
    M0 = partial_subset_sum(S, i-1, t)
    # Et la meilleure solution contenant S[i]
    if t - S[i] >= 0:
        M1 = S[i] + partial_subset_sum(S, i-1, t - S[i])
    else:
        M1 = 0
    return max(M0, M1)

def subset_sum(S, t):
    M = partial_subset_sum(S, len(S) - 1, t)
    # On renvoie aussi le maximum atteignable <= t 
    # (solution du problème d'optimisation)
    return M == t, M

In [6]:
subset_sum(S, 22), subset_sum(S, 10), subset_sum(S, 56)

((True, 22), (False, 9), (True, 56))

## Partie 3: programmation dynamique

In [7]:
def partial_subset_sum_memo(S, i, t):
    '''
    Solution d'un subset-sum tronqué au i-ème élément, par mémoisation
    '''
    # Si on considère la sous-liste vide, on ne peut obtenir que 0
    if i < 0:
        return [0]
    # Les sous-ensembles ne contenant pas S[i]
    P0 = partial_subset_sum_memo(S, i-1, t)
    # Et les sous-ensembles contenant S[i]
    P1 = [S[i] + s for s in P0 if s + S[i] <= t]
    # On fusionne les listes P0 et P1, avec la même procédure que le tri fusion
    P = [0]
    while P0 and P1:
        if P0[0] < P1[0]:
            c = P0.pop(0)
        else:
            c = P1.pop(0)
        # Inutile d'avoir des dupliqués dans la liste
        if c != P[-1]:
            P.append(c)
    # S'il reste des éléments dans P0 ou P1, on les ajoute
    P.extend(P0 + P1)
    return P

def subset_sum_memo(S, t):
    P = partial_subset_sum_memo(S, len(S) - 1, t)
    # On renvoie aussi le maximum atteignable <= t 
    # (solution du problème d'optimisation)
    return P[-1] == t, P[-1]

In [8]:
subset_sum_memo(S, 22), subset_sum_memo(S, 10), subset_sum_memo(S, 56)

((True, 22), (False, 9), (True, 56))

In [9]:
def partial_subset_sum_dyn(S, i, t):
    '''
    Solution d'un subset-sum tronqué au i-ème élément, par programmation dynamique
    '''
    P = [0]
    for j in range(i+1):
        # Les sous-ensembles contenant S[j]
        P1 = [s + S[j] for s in P if s + S[j] <= t]
        # On fusionne les listes P et P1
        newP = [0]
        while P and P1:
            if P[0] < P1[0]:
                c = P.pop(0)
            else:
                c = P1.pop(0)
            # Inutile d'avoir des dupliqués dans la liste
            if c != newP[-1]:
                newP.append(c)
        # S'il reste des éléments dans P0 ou P1, on les ajoute
        newP.extend(P + P1)
        P = newP
    return P[-1]

def subset_sum_dyn(S, t):
    M = partial_subset_sum_dyn(S, len(S) - 1, t)
    # On renvoie aussi le maximum atteignable <= t 
    # (solution du problème d'optimisation)
    return M == t, M

In [10]:
subset_sum_dyn(S, 22), subset_sum_dyn(S, 10), subset_sum_dyn(S, 56)

((True, 22), (False, 9), (True, 56))

In [11]:
import random
S = [random.randrange(2**32) for _ in range(2^16)]
t = random.randrange(2**36)

In [12]:
%timeit subset_sum(S, t)
%timeit subset_sum_memo(S, t)
%timeit subset_sum_dyn(S, t)

10 loops, best of 3: 170 ms per loop
1 loop, best of 3: 6.79 s per loop
1 loop, best of 3: 6.74 s per loop


Décevant, n'est-ce pas ? C'est le coût à payer quand on utilise beaucoup de mémoire: une solution qui a l'air plus intelligente en principe, peut être pénalisée par une plus grande complexité des accés à la mémoire.

Nos versions mémoisées et par programmation dynamique pourraient être optimisées, mais obtenir un algorithme vraiment rapide est un vrai travail de niveau recherche (par exemple en optimisation ou cryptologie).